# Multi-material NEB Pipeline (Restartable, with time-logged steps)

This notebook builds a **restartable, multi-material NEB workflow** for
surface diffusion of simple metal anodes (Li, Na, K, Mg, Al).

Key features:

- All slabs are built **directly from ASE** (`surface()`), no external files.
- Uses **`ase.mep.NEB`**.
- Each important step (initial relaxation, final relaxation, NEB) is
  tracked in a small `status.json` file so you can **restart after kernel
  shutdown** without redoing finished work.
- Relaxations and NEB run step-by-step with **text logging**, including
  **elapsed wall-clock time per step**, so you can see progress.


## 1. Environment and dependencies

Requirements:

- Python environment with **ASE** and **GPAW** working.
- No extra Python packages beyond the standard library.

On a cluster, you might need (example, adjust to your system):

```bash
module load gpaw ase
```

Then start Jupyter and run this notebook.


In [6]:
# 2. Imports
from ase.build import surface
from ase.io import write, read
from ase import Atom
from ase.optimize import BFGS, FIRE
from ase.mep import NEB
from gpaw import GPAW, PW
from ase.parallel import world
from ase.constraints import FixAtoms

import numpy as np
import os
import json
import time

print("Imports OK")


Imports OK


## 3. Global parameters

Edit this cell to change:

- Which materials and surfaces to study.
- Slab size, vacuum, and adatom height.
- GPAW and NEB convergence settings.

All slabs are generated from scratch; no external structure files are needed.


In [7]:
# 3. Global parameters

materials = {
    "Li": ((1, 1, 0),  "Li bcc (110)"),
    "Na": ((1, 1, 0),  "Na bcc (110)"),
}

layers = 4
supercell = (3, 3, 1)
vacuum = 10.0
adatom_height = 2.0

relax_slab = False
relax_initial_final = True

kp = 2
PWc = 200
xc = "PBE"
slab_fmax = 0.05
initial_final_fmax = 0.05
neb_fmax = 0.10

n_middle_images = 5
neb_max_steps = 300

output_dir = "multi_neb_results_restartable_time"
os.makedirs(output_dir, exist_ok=True)
status_path = os.path.join(output_dir, "status.json")

print(f"Output directory: {output_dir}")
print(f"Status file: {status_path}")


Output directory: multi_neb_results_restartable_time
Status file: multi_neb_results_restartable_time/status.json


## 4. Helper functions

Defines:
- Slab building and adatom placement.
- GPAW calculator factory.
- Status load/save.
- Restart-aware relaxation with time-logged steps.


In [8]:
def build_slab(symbol, indices):
    slab = surface(symbol, indices, layers=layers, vacuum=vacuum)
    slab = slab.repeat(supercell)
    return slab


def choose_two_top_sites(slab, initial_dz=0.5):
    positions = slab.get_positions()
    z = positions[:, 2]
    z_max = z.max()

    for dz in [initial_dz, 1.0, 2.0]:
        top_indices = np.where(z > z_max - dz)[0]
        if len(top_indices) >= 2:
            break
    else:
        raise RuntimeError(f"Not enough top-layer atoms found (max z = {z_max:.3f}).")

    i1 = top_indices[0]
    p1 = positions[i1]

    best_j = None
    best_d = 1e9
    for j in top_indices[1:]:
        dp = positions[j] - p1
        d_xy = np.linalg.norm(dp[:2])
        if 1e-5 < d_xy < best_d:
            best_d = d_xy
            best_j = j

    if best_j is None:
        raise RuntimeError("Could not find a suitable neighbouring top-layer site.")

    p2 = positions[best_j]
    return p1, p2, z_max


def apply_bottom_fix(slab, fraction=0.5):
    z = slab.get_positions()[:, 2]
    z_min, z_max = z.min(), z.max()
    z_cut = z_min + fraction * (z_max - z_min)
    mask = z < z_cut
    slab.set_constraint(FixAtoms(mask=mask))
    return slab


def make_adatom_state(base_slab, symbol, site_pos, z_max):
    slab = base_slab.copy()
    slab = apply_bottom_fix(slab, fraction=0.5)
    adatom = Atom(symbol, [site_pos[0], site_pos[1], z_max + adatom_height])
    slab.append(adatom)
    return slab


def make_gpaw(label):
    return GPAW(
        mode=PW(PWc),
        xc=xc,
        kpts=(kp, kp, 1),
        convergence={"eigenstates": 5e-6, "density": 1e-4},
        txt=os.path.join(output_dir, f"{label}.txt"),
    )


def load_status():
    if not os.path.exists(status_path):
        return {}
    try:
        with open(status_path, "r") as f:
            return json.load(f)
    except Exception:
        return {}


def save_status(status):
    with open(status_path, "w") as f:
        json.dump(status, f, indent=2)


def relax_structure(atoms, label, fmax, allow_restart=True):
    """Relax a structure with BFGS until max force < fmax.

    - Writes a `.traj` file to disk.
    - If `allow_restart` and `<label>.traj` exists, restart from the
      last saved frame instead of the raw input geometry.
    - Logs step number, max force, and elapsed time.
    """
    traj_path = os.path.join(output_dir, f"{label}.traj")

    if allow_restart and os.path.exists(traj_path):
        if world.rank == 0:
            print(f"[{label}] Restarting from existing trajectory: {traj_path}")
        try:
            atoms = read(traj_path, index=-1)
        except Exception as e:
            if world.rank == 0:
                print(f"[{label}] Could not read existing traj, starting fresh. Reason: {e}")

    atoms.calc = make_gpaw(label)
    dyn = BFGS(
        atoms,
        logfile=os.path.join(output_dir, f"{label}.log"),
        trajectory=traj_path,
    )

    step = 0
    t0 = time.time()
    if world.rank == 0:
        print(f"[{label}] Starting relaxation (target fmax = {fmax:.3f} eV/Å)")

    for _ in dyn.irun(fmax=fmax):
        step += 1
        if world.rank == 0:
            elapsed = time.time() - t0
            forces = atoms.get_forces()
            maxf = float(np.abs(forces).max())
            print(f"[{label}] step {step:4d}, max |F| = {maxf:.3f} eV/Å, elapsed = {elapsed:7.1f} s")

    if world.rank == 0:
        total = time.time() - t0
        print(f"[{label}] Relaxation finished in {step} steps, total time = {total:.1f} s.")
    return atoms

print("Helper functions defined.")


Helper functions defined.


## 5. NEB driver (restart-aware, per-step timing)

Runs NEB for a single material, using `status.json` to avoid redoing work,
        
and logs **each NEB optimization step** with elapsed time.


In [9]:
def run_neb_for_material(symbol, indices, description=""):
    """Run the NEB workflow for one material, restart-aware.

    Uses status.json to avoid redoing completed stages.
    Logs every NEB step with approximate max force and elapsed time.
    """
    status = load_status()
    mat_status = status.get(symbol, {
        "initial_relaxed": False,
        "final_relaxed": False,
        "neb_done": False,
    })

    if world.rank == 0:
        print("\n" + "=" * 60)
        print(f"Running NEB for {symbol}: {description}")
        print("Current status:", mat_status)
        print("=" * 60)

    # 1. Build slab
    slab = build_slab(symbol, indices)

    if world.rank == 0:
        write(os.path.join(output_dir, f"{symbol}-slab.xyz"), slab)

    # 2. Choose diffusion sites
    p1, p2, z_max = choose_two_top_sites(slab)

    # 3. Build initial and final adatom states
    initial = make_adatom_state(slab, symbol, p1, z_max)
    final = make_adatom_state(slab, symbol, p2, z_max)

    # 4. Relax initial and final states if requested and not already done
    if relax_initial_final:
        if not mat_status.get("initial_relaxed", False):
            if world.rank == 0:
                print(f"{symbol}: relaxing initial adatom configuration...")
            initial = relax_structure(initial, f"{symbol}-initial", initial_final_fmax, allow_restart=True)
            mat_status["initial_relaxed"] = True
            status[symbol] = mat_status
            save_status(status)
        else:
            if world.rank == 0:
                print(f"{symbol}: initial state already relaxed (skipping).")
            initial = read(os.path.join(output_dir, f"{symbol}-initial.traj"), index=-1)

        if not mat_status.get("final_relaxed", False):
            if world.rank == 0:
                print(f"{symbol}: relaxing final adatom configuration...")
            final = relax_structure(final, f"{symbol}-final", initial_final_fmax, allow_restart=True)
            mat_status["final_relaxed"] = True
            status[symbol] = mat_status
            save_status(status)
        else:
            if world.rank == 0:
                print(f"{symbol}: final state already relaxed (skipping).")
            final = read(os.path.join(output_dir, f"{symbol}-final.traj"), index=-1)

    if world.rank == 0:
        write(os.path.join(output_dir, f"{symbol}-initial.xyz"), initial)
        write(os.path.join(output_dir, f"{symbol}-final.xyz"), final)

    # 5. NEB
    if mat_status.get("neb_done", False):
        if world.rank == 0:
            print(f"{symbol}: NEB already marked as done. Skipping NEB run.")
        energies_file = os.path.join(output_dir, f"{symbol}-NEB-energies.dat")
        if os.path.exists(energies_file):
            data = np.loadtxt(energies_file, comments="#")
            energies = data[:, 1]
            ref = min(energies[0], energies[-1])
            barrier = energies.max() - ref
            return barrier
        else:
            if world.rank == 0:
                print(f"{symbol}: NEB marked done but energies file missing. Rerunning NEB.")
            mat_status["neb_done"] = False
            status[symbol] = mat_status
            save_status(status)

    images = [initial]
    images += [initial.copy() for _ in range(n_middle_images)]
    images += [final]

    neb = NEB(images, climb=True)
    neb.interpolate()

    for i, img in enumerate(images):
        img.calc = make_gpaw(f"{symbol}-NEB-img{i}")

    if world.rank == 0:
        print(f"{symbol}: starting NEB optimization with {len(images)} images "
              f"(target fmax = {neb_fmax:.3f} eV/Å)")

    opt = FIRE(
        neb,
        logfile=os.path.join(output_dir, f"{symbol}-NEB.log"),
        trajectory=os.path.join(output_dir, f"{symbol}-NEB.traj"),
    )

    step = 0
    t0 = time.time()
    for _ in opt.irun(fmax=neb_fmax, steps=neb_max_steps):
        step += 1
        if world.rank == 0:
            elapsed = time.time() - t0
            try:
                neb_forces = neb.get_forces()
                maxf = float(np.abs(neb_forces).max())
            except Exception:
                maxf = float("nan")
            print(f"[{symbol} NEB] step {step:4d}, NEB max |F| = {maxf:.3f} eV/Å, elapsed = {elapsed:7.1f} s")

    if world.rank == 0:
        total = time.time() - t0
        print(f"[{symbol} NEB] optimization finished after {step} steps, total time = {total:.1f} s.")

    energies = np.array([img.get_potential_energy() for img in images])
    ref = min(energies[0], energies[-1])
    barrier = energies.max() - ref

    if world.rank == 0:
        print(f"{symbol}: diffusion barrier ≈ {barrier:.3f} eV")
        energies_file = os.path.join(output_dir, f"{symbol}-NEB-energies.dat")
        with open(energies_file, "w") as f:
            f.write("# image   energy_eV\n")
            for i, E in enumerate(energies):
                f.write(f"{i:3d}  {E:15.8f}\n")

    mat_status["neb_done"] = True
    status[symbol] = mat_status
    save_status(status)

    return barrier

print("NEB driver function defined (with time-logged steps).")

NEB driver function defined (with time-logged steps).


## 6. Run NEB for all materials

You can re-run this cell multiple times. It uses `status.json` and the
relax/NEB outputs to skip work that is already done.


In [10]:
summary_path = os.path.join(output_dir, "diffusion_barriers_summary.dat")

if world.rank == 0:
    sf = open(summary_path, "w")
    sf.write("# symbol   barrier_eV\n")
else:
    sf = None

for symbol, (indices, desc) in materials.items():
    try:
        barrier = run_neb_for_material(symbol, indices, description=desc)
        if world.rank == 0:
            sf.write(f"{symbol:6s}  {barrier:10.4f}\n")
    except Exception as e:
        if world.rank == 0:
            print(f"{symbol}: FAILED – {e}")
            sf.write(f"{symbol:6s}  FAILED\n")

if sf is not None:
    sf.close()
    if world.rank == 0:
        print("\nAll done. Summary written to:", summary_path)



Running NEB for Li: Li bcc (110)
Current status: {'initial_relaxed': True, 'final_relaxed': True, 'neb_done': True}
Li: initial state already relaxed (skipping).
Li: final state already relaxed (skipping).
Li: NEB already marked as done. Skipping NEB run.

Running NEB for Na: Na bcc (110)
Current status: {'initial_relaxed': True, 'final_relaxed': True, 'neb_done': True}
Na: initial state already relaxed (skipping).
Na: final state already relaxed (skipping).
Na: NEB already marked as done. Skipping NEB run.

All done. Summary written to: multi_neb_results_restartable_time/diffusion_barriers_summary.dat
